# [Build a Chatbot](https://python.langchain.com/v0.2/docs/tutorials/chatbot/)
[Overview](https://python.langchain.com/v0.2/docs/tutorials/chatbot/#overview "Direct link to Overview")
---------------------------------------------------------------------------------------------------------

我們將探討如何設計和實現一個由 LLM 驅動的聊天機器人的示例。這個聊天機器人將能夠進行對話並記住之前的互動。

請注意，我們構建的這個聊天機器人只會使用語言模型進行對話。不過，你可能還會對其他相關概念感興趣：

-   [Conversational RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/): 讓聊天機器人可以基於外部數據源進行對話
-   [Agents](https://python.langchain.com/v0.2/docs/tutorials/agents/): 構建能夠執行操作的聊天機器人

本教程將涵蓋基本內容，這些內容對後面更高級的主題也會有幫助，但如果你有興趣，也可以直接跳到那裡學習。

In [13]:

# import os
# from langchain_openai import AzureChatOpenAI
# model = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
# )
# model

from langchain_ollama import ChatOllama
model_name = "llama3.1"
# model_name = "jcai/taide-lx-7b-chat"
model = ChatOllama(model=model_name)
model


ChatOllama(model='llama3.1')

In [14]:
from langchain_core.messages import HumanMessage

In [61]:

response = model.invoke([HumanMessage(content="Hi, 我是品至.")])
response.content

'Nice to meet you, 品至! How can I assist you today?'

模型本身並沒有任何狀態的概念。例如，如果你問一個後續問題：

In [62]:
'''
Trace Sample:
https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r
'''
response = model.invoke([HumanMessage(content="請問我的名字是什麼？")])
response.content


'對不起，我們第一次見面就沒有交流過你的名字。'

In [63]:
from langchain_core.messages import AIMessage

我們來看一下這個示例 [LangSmith trace](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)。

我們可以看到，模型並沒有將之前的對話輪次考慮在內，因此無法回答問題。這樣的聊天機器人體驗非常糟糕！

為了解決這個問題，我們需要將整個對話歷史傳遞給模型。讓我們看看這樣做會發生什麼：

In [64]:
response = model.invoke(
    [
        HumanMessage(content="Hi ,我叫品至"),
        AIMessage(content="你好，品至！有什麼我可以幫助您的嗎？"),
        HumanMessage(content="我的名字是什麼？"),
    ]
)
response.content

'你的名字是品至呦！'

現在我們可以看到，我們得到了更好的回應！

這就是支持聊天機器人進行對話交互的基本原理。那麼，我們該如何最佳化地實現這一點呢？

[Message History](https://python.langchain.com/v0.2/docs/tutorials/chatbot/#message-history "Direct link to Message History")
------------------------------------------------------------------------------------------------------------------------------

我們可以使用 Message History 類來封裝模型，使其具有狀態。這個類將跟蹤模型的輸入和輸出，並將它們存儲在某個數據庫中。未來的交互將加載這些消息，並將它們作為輸入的一部分傳遞給鏈。讓我們看看如何使用這個方法！

首先，請確保安裝 `langchain-community`，因為我們將使用其中的一個集成來存儲消息歷史記錄。

In [9]:
# ! pip3 install langchain_community

接下來，我們可以導入相關的類並設置我們的鏈，這個鏈將封裝模型並添加消息歷史記錄。關鍵的一部分是我們傳入的 get_session_history 函數。這個函數應該接受一個 session_id，並返回一個 Message History 對象。session_id 用於區分不同的對話，並應該在調用新的鏈時作為配置的一部分傳入（我們將展示如何操作）。

In [2]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [3]:
config = {"configurable": {"session_id": "abc2"}}

In [67]:
response = with_message_history.invoke(
    [HumanMessage(content="哈囉!我叫品至.")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'你好！很高興遇到你，品至！想聊什麼嗎？'

In [68]:
response = with_message_history.invoke(
    [HumanMessage(content="我叫什麼?")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'妳是品至!妳剛剛自己說的哦!'

太棒了！我們的聊天機器人現在能記住與我們相關的信息。如果我們更改配置中的 session_id，可以看到對話會從頭開始。
這樣，我們就能支持聊天機器人同時與多個用戶進行對話了！

目前，我們只是為模型添加了一個簡單的持久化層。接下來，我們可以通過添加提示模板，使聊天機器人變得更加複雜和個性化。

## [Prompt templates](https://python.langchain.com/v0.2/docs/tutorials/chatbot/#prompt-templates "Direct link to Prompt templates")

---------------------------------------------------------------------------------------------------------------------------------

提示模板有助於將原始的使用者資訊轉換成語言模型 (LLM) 能夠處理的格式。在這個案例中，原始的使用者輸入只是一則訊息，我們將其傳遞給 LLM。現在，讓我們使這個過程稍微複雜一些。首先，加入一則帶有自訂指令的系統訊息（但仍然以訊息作為輸入）。接下來，我們將添加更多除了訊息以外的輸入內容。

首先，讓我們加入一則系統訊息。為此，我們將創建一個 ChatPromptTemplate。我們將利用 `MessagesPlaceholder` 來傳遞所有訊息。

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一個有用的助手. 盡你所能回答所有的問題.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [70]:
response = chain.invoke({"messages": [HumanMessage(content="hi! 我叫品至.")]})

response.content

'你好！很高興認識你，品至！我是你的助手，隨時準備幫助你解答任何疑問或進行任何討論。您有什么需要我的幫助的嗎？'

In [5]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [6]:
config = {"configurable": {"session_id": "abc5"}}

In [73]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! 我叫珍妮.")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'欢迎珍妮！很高兴认识你！你有什么问题或话想说吗？'

In [74]:
response = with_message_history.invoke(
    [HumanMessage(content="我的名字是什麼？")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'你的名字是珍妮（Jenny）！'

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一個有用的助手. 盡你所能回答所有的問題. 確保使用語言: {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

請注意，這稍微改變了輸入類型------我們現在不再傳遞訊息列表，而是傳遞一個包含 `messages` 鍵的字典，其中該鍵包含了一個訊息列表。

In [76]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! 我是愛麗絲.")], "language": "繁體中文"}
)

response.content

'你好！我是你的助手，很高興遇見你，艾麗絲（Alice）！怎麼了？有什麼事需要幫忙的嗎？'

我們現在可以像之前一樣，將這個包裹在相同的訊息歷史對象中。

In [15]:
with_message_history = RunnableWithMessageHistory(
    chain, # prompt | model
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! 我是高進.")], "language": "繁體中文"},
    config=config,
)
response.content


Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [84]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="我叫什麼?")], "language": "繁體中文"},
    config=config,
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'你刚才自己告訴我的，你叫做"高進"！'

# [Managing Conversation History](https://python.langchain.com/v0.2/docs/tutorials/chatbot/#managing-conversation-history "Direct link to Managing Conversation History")

------------------------------------------------------------------------------------------------------------------------------------------------------------------------

在構建聊天機器人時，一個重要的概念是如何管理對話歷史記錄。如果不加以管理，訊息列表會無限增長，最終可能會超出LLM的上下文窗口。因此，添加一個限制所傳遞訊息大小的步驟是很重要的。

特別要注意的是，這個步驟應該在提示模板**之前**，但在從訊息歷史記錄中加載先前訊息**之後**執行。

我們可以通過在提示之前添加一個簡單的步驟來適當修改 `messages` 鍵，然後將這個新鏈條包裹在訊息歷史記錄類中。

LangChain 提供了一些內建的輔助工具來[管理訊息列表](https://python.langchain.com/v0.2/docs/how_to/#messages)。在這個案例中，我們將使用 [trim_messages](https://python.langchain.com/v0.2/docs/how_to/trim_messages/) 輔助工具來減少傳送給模型的訊息數量。這個工具允許我們指定要保留的代幣數量，以及其他參數，如是否始終保留系統訊息以及是否允許部分訊息。

In [85]:
from langchain_community.chat_models import ChatOllama
model = ChatOllama(model="llama3.1")
model
# ! pip3 install transformers

ChatOllama(model='llama3.1')

In [108]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=120,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="你是個有用的助手."),
    HumanMessage(content="hi! 我叫品至."),
    AIMessage(content="Hi!"),
    HumanMessage(content="我喜歡香草冰淇淋!"),
    AIMessage(content="很好!"),
    HumanMessage(content="2 + 2 答案是什麼?"),
    AIMessage(content="4"),
    HumanMessage(content="謝謝"),
    AIMessage(content="不客氣!"),
    HumanMessage(content="有趣嗎?"),
    AIMessage(content="有趣!"),
]

trimmer.invoke(messages)

[SystemMessage(content='你是個有用的助手.'),
 HumanMessage(content='我喜歡香草冰淇淋!'),
 AIMessage(content='很好!'),
 HumanMessage(content='2 + 2 答案是什麼?'),
 AIMessage(content='4'),
 HumanMessage(content='謝謝'),
 AIMessage(content='不客氣!'),
 HumanMessage(content='有趣嗎?'),
 AIMessage(content='有趣!')]

要在我們的鏈條中使用它，我們只需要在將 `messages` 輸入傳遞給提示模板之前運行修剪器。

現在，如果我們嘗試詢問模型我們的名字，它可能不會知道，因為我們已經修剪了那部分的聊天歷史記錄。

In [109]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="我的名字是什麼?")],
        "language": "繁體中文",
    }
)
response.content

'你沒有告訴我你的名字，我也不知道。這裡的對話才剛開始，你可以先自我介紹一下！'

但是，如果我們詢問最近幾條訊息中的資訊，它還是會記得的：

In [111]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="剛剛我問了什麼？")],
        "language": "繁體中文",
    }
)
response.content

'你剛才問的是 "2 + 2 答案是什麼?" 的問題。然後你也跟我說了 "謝謝"，我回覆了 "不客氣!"，然後你又問了 "有趣嗎?"！'

現在，讓我們將這個包裹在訊息歷史記錄中。

In [9]:
with_message_history = RunnableWithMessageHistory(
    chain, # trimer | prompt | model
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [116]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="我的名字是什麼?")],
        "language": "繁體中文",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'我不知道你的名字!你能告訴我你的名字嗎?'

正如預期的那樣，我們最初提到名字的訊息已經被修剪掉了。此外，現在聊天歷史記錄中有兩條新訊息（我們的最新問題和最新回應）。這意味著我們對話歷史中原本可訪問的更多資訊也不再可用！在這個例子中，我們最初的數學問題也已經從歷史記錄中被修剪掉了，所以模型已經不再知道它的存在。

# [Streaming](https://python.langchain.com/v0.2/docs/tutorials/chatbot/#streaming "Direct link to Streaming")
------------------------------------------------------------------------------------------------------------

現在我們已經有了一個功能完善的聊天機器人。然而，對於聊天機器人應用程序來說，一個*非常*重要的用戶體驗考量是串流（Streaming）。由於LLMs有時可能需要一段時間才能回應，因此為了提升用戶體驗，大多數應用程序會在生成每個代幣時立即串流回傳，這樣用戶就能看到進展。

其實這個功能非常容易實現！

所有的鏈條都提供了一個 `.stream` 方法，使用訊息歷史記錄的鏈條也不例外。我們只需使用該方法即可獲得串流回應。

In [ ]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="HI. 我是高進. 請跟我說我一個短笑話. 最後跟我說好笑在哪裡")],
        "language": "繁體中文",
    },
    config=config,
):
    print(r.content, end="")

# [Next Steps](https://python.langchain.com/v0.2/docs/tutorials/chatbot/#next-steps "Direct link to Next Steps")
---------------------------------------------------------------------------------------------------------------

現在您已經了解如何在 LangChain 中創建聊天機器人的基礎知識，您可能會對一些更高級的教程感興趣：

-   [Conversational RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/): 使聊天機器人能夠基於外部數據源提供對話體驗
-   [Agents](https://python.langchain.com/v0.2/docs/tutorials/agents/): 構建能夠執行操作的聊天機器人

如果您想深入了解一些具體內容，以下內容值得查看：

-   [Streaming](https://python.langchain.com/v0.2/docs/how_to/streaming/): 串流對於聊天應用至關重要
-   [How to add message history](https://python.langchain.com/v0.2/docs/how_to/message_history/): 更深入地了解與消息歷史相關的一切
-   [How to manage large message history](https://python.langchain.com/v0.2/docs/how_to/trim_messages/): 更多管理大型聊天歷史記錄的技巧